# Cuaderno 15: Trabajando con datos reales

Revisaremos en este cuaderno algunos problemas que pueden presentarse al procesar datos en la práctica, y cómo podemos abordarlos sistemáticamente con las herramientas disponibles en Python y `pandas`.

Empezamos por importar los módulos de `pandas`y `numpy`:

In [1]:
# importar pandas y NumPy
import numpy as np
import pandas as pd

## Cruzando conjuntos de datos grandes

Retomemos el ejemplo del Cuaderno 14. Vamos a trabajar con datos del registro de defunciones generales del Ecuador en el año 2019, obtenido del sitio web del Instituto Nacional de Estadística y Censos del Ecuador (INEC) <https://www.ecuadorencifras.gob.ec/defunciones-generales-2019/>. De este archivo en formato `csv` nos interesan solamente algunas variables:
* `sexo`: sexo de la persona fallecida (1: Masculino, 2: Femenino)
* `prov_fall`: código de la provincia en la que se registró el fallecimiento
* `cant_fall`: código del cantón en el que se registró en fallecimiento
* `mor_vio`: causa del fallecimiento, en el caso de las muertes violentas (número del 1 al 6, en blanco significa muerte natural).

Importamos estas columnas, sustituyendo los espacios en blanco por `NaN`:

In [2]:
# importar datos del registro de defunciones de 2019 del INEC
dfmuertes2019 = pd.read_csv('BDD_EDG_2019.csv', sep=';', 
                            usecols=['sexo', 'prov_fall', 'cant_fall', 'mor_viol'], 
                            na_values=' ') # espacios en blanco se tratarán como NaN
display(dfmuertes2019)

,sexo,prov_fall,cant_fall,mor_viol
0,1,9,901,1.0
1,1,17,1701,NaN
2,1,17,1701,2.0
3,1,17,1701,NaN
4,1,9,901,NaN
...,...,...,...,...
75350,2,17,1701,NaN
75351,2,9,906,NaN
75352,2,5,501,2.0
75353,2,9,901,NaN


Notar que el archivo tiene 75355 filas. Renombramos las columnas de `dfmuertes2019` con nombres más adecuados para nuestro ejemplo. Reemplazaremos además la columna `mor_viol` por una columna `violenta` en la que se indique (con 0 o 1) si se trata de una muerta violenta o no:

In [3]:
# sustituir nombres de las columnas
dfmuertes2019.rename(columns= {'prov_fall' : 'cod_provincia', 'cant_fall' : 'cod_canton', 'mor_viol' : 'violenta'}, 
                     inplace= True)
# sustituir valores de NaN por cero
dfmuertes2019.fillna(0, inplace=True)
# sustituir valores mayores a cero en la columna violenta por 1's
dfmuertes2019.loc[dfmuertes2019['violenta']>0,'violenta'] = 1
dfmuertes2019= dfmuertes2019.astype({'violenta':'int'})
display(dfmuertes2019)

,sexo,cod_provincia,cod_canton,violenta
0,1,9,901,1
1,1,17,1701,0
2,1,17,1701,1
3,1,17,1701,0
4,1,9,901,0
...,...,...,...,...
75350,2,17,1701,0
75351,2,9,906,0
75352,2,5,501,1
75353,2,9,901,0


En la columna `cod_canton`, cada registro tiene un código (un número entero) que identifica el cantón del fallecimiento. Para incorporar los nombres de los cantones y su población en el 2019, importaremos estos datos desde el archivo `proyeccion_cantonal_total_2010-2020.xlsx` al DataFrame `dfcantones`. El archivo fue también descargado del sitio web del Instituto Nacional de Estadística y Censos del Ecuador (INEC) <https://www.ecuadorencifras.gob.ec/proyecciones-poblacionales/>. 

In [4]:
# importar datos de proyección poblacional por cantones del INEC
dfcantones = pd.read_excel('proyeccion_cantonal_total_2010-2020.xlsx', skiprows=2,
                            usecols='A:B,L') 
display(dfcantones)

,Código,Nombre de canton,2019
0,101,CUENCA,625775
1,102,GIRON,13074
2,103,GUALACEO,48702
3,104,NABON,17250
4,105,PAUTE,28985
...,...,...,...
219,2402,LIBERTAD,115952
220,2403,SALINAS,92017
221,9001,LAS GOLONDRINAS,7370
222,9003,MANGA DEL CURA,26061


Nuevamente, cambiamos los nombres de las columnas de `dfcantones` por valores más adecuados:

In [5]:
dfcantones.rename(columns={'Código' : 'cod_canton', 'Nombre de canton' : 'canton', 
                           2019 : 'poblacion'}, inplace= True)
display(dfcantones)

,cod_canton,canton,poblacion
0,101,CUENCA,625775
1,102,GIRON,13074
2,103,GUALACEO,48702
3,104,NABON,17250
4,105,PAUTE,28985
...,...,...,...
219,2402,LIBERTAD,115952
220,2403,SALINAS,92017
221,9001,LAS GOLONDRINAS,7370
222,9003,MANGA DEL CURA,26061


Empleando la función `merge`, cruzamos la información de los DataFrames `dfmuertes2019` y `dfcantones` para obtener el DataFrame `dfmuertes2019ext`, que contiene el registro de defunciones con información del nombre del cantón y su población. 

In [6]:
dfmuertes2019ext= pd.merge(dfmuertes2019, dfcantones, on='cod_canton')
display(dfmuertes2019ext) 

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
0,1,9,901,1,GUAYAQUIL,2698077
1,1,9,901,0,GUAYAQUIL,2698077
2,1,9,901,0,GUAYAQUIL,2698077
3,1,9,901,1,GUAYAQUIL,2698077
4,1,9,901,0,GUAYAQUIL,2698077
...,...,...,...,...,...,...
75215,2,16,1604,0,ARAJUNO,7989
75216,1,16,1603,0,SANTA CLARA,4110
75217,1,16,1603,1,SANTA CLARA,4110
75218,2,16,1603,0,SANTA CLARA,4110


Notar que en este proceso se pierden 75355 - 75220= 135 registros. Esto probablemente puede deberse a que ciertas entradas del DataFrame `dfmuertes2019` tenían códigos de cantón que no están registrados en la tabla `dfcantones`.
Para verificar esto y encontrar cuáles son los códigos con problema, volvamos a cruzar las tablas, pero usando esta vez el tipo `left join` para la aritmética de conjuntos. Recordemos que este tipo conserva todas las filas del primer DataFrame, e incorpora valores `NaN` cuando no hay registros correspondientes en el segundo DataFrame: 

In [7]:
dfmuertes2019ext= pd.merge(dfmuertes2019, dfcantones, on='cod_canton', how='left')
display(dfmuertes2019ext)

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
0,1,9,901,1,GUAYAQUIL,2698077.0
1,1,17,1701,0,QUITO,2735987.0
2,1,17,1701,1,QUITO,2735987.0
3,1,17,1701,0,QUITO,2735987.0
4,1,9,901,0,GUAYAQUIL,2698077.0
...,...,...,...,...,...,...
75350,2,17,1701,0,QUITO,2735987.0
75351,2,9,906,0,DAULE,168144.0
75352,2,5,501,1,LATACUNGA,202878.0
75353,2,9,901,0,GUAYAQUIL,2698077.0


Este DataFrame contiene el mismo número de registros que el DataFrame original `dfmuertes2019`. Podemos averiguar si una columna de un DataFrame contiene valores `NaN` empleando el método `.isnull().any()`:

In [8]:
display(dfmuertes2019ext.isnull().any())

sexo             False
cod_provincia    False
cod_canton       False
violenta         False
canton            True
poblacion         True
dtype: bool

Conforme a nuestras suposiciones, hay registros (filas) que tienen valores de `NaN` para el nombre del cantón y su población. Podemos extraer estos registros usando el método `.isnull()` como condición de filtrado sobre la columna `canton` de `dfmuertes2019ext`:

In [9]:
display(dfmuertes2019ext[dfmuertes2019ext['canton'].isnull()])

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
215,1,23,2302,1,NaN,NaN
9062,1,23,2302,1,NaN,NaN
9612,1,23,2302,0,NaN,NaN
9627,1,23,2302,0,NaN,NaN
9629,1,23,2302,0,NaN,NaN
...,...,...,...,...,...,...
72251,2,23,2302,1,NaN,NaN
72306,2,23,2302,0,NaN,NaN
72364,2,23,2302,0,NaN,NaN
72387,2,23,2302,0,NaN,NaN


Como lo esperábamos, obtenemos exactamente 135 filas sin nombre de cantón. Aparentemente, todas estas filas tienen el código de cantón 2302. Sin embargo, para asegurarnos de que no existan otros códigos con problemas, agrupamos este último DataFrame por la columna `cod_canton` y contamos las filas:

In [10]:
display(dfmuertes2019ext[dfmuertes2019ext['canton'].isnull()].groupby('cod_canton').count())

,sexo,cod_provincia,violenta,canton,poblacion
cod_canton,,,,,
2302,135,135,135,0,0


Corroboramos que, efectivamente, el único código faltante es el código 2302, que además está asociado al código de provincia 23, correspondiente a la provincia de Santo Domingo de los Tsáchilas. 

Al consultar información al respecto, podemos verificar que esta provincia tiene dos cantones: Santo Domingo y La Concordia. De ellos, el cantón La Concordia no está registrado en el archivo `proyeccion_cantonal_total_2010-2020.xlsx`, posiblemente por ser de reciente creación. La población estimada de este cantón es de 42924 habitantes.

Agregamos el registro correspondiente al DataFrame `dfcantones` y reintentamos cruzar la información:

In [11]:
dfcantones= dfcantones.append({'cod_canton' :  2302 , 'canton' : 'LA CONCORDIA', 'poblacion' : 42924}, 
                              ignore_index=True)
display(dfcantones)

dfmuertes2019ext= pd.merge(dfmuertes2019, dfcantones, on='cod_canton')
display(dfmuertes2019ext)

,cod_canton,canton,poblacion
0,101,CUENCA,625775
1,102,GIRON,13074
2,103,GUALACEO,48702
3,104,NABON,17250
4,105,PAUTE,28985
...,...,...,...
220,2403,SALINAS,92017
221,9001,LAS GOLONDRINAS,7370
222,9003,MANGA DEL CURA,26061
223,9004,EL PIEDRERO,7651


,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
0,1,9,901,1,GUAYAQUIL,2698077
1,1,9,901,0,GUAYAQUIL,2698077
2,1,9,901,0,GUAYAQUIL,2698077
3,1,9,901,1,GUAYAQUIL,2698077
4,1,9,901,0,GUAYAQUIL,2698077
...,...,...,...,...,...,...
75350,2,16,1604,0,ARAJUNO,7989
75351,1,16,1603,0,SANTA CLARA,4110
75352,1,16,1603,1,SANTA CLARA,4110
75353,2,16,1603,0,SANTA CLARA,4110


En esta ocasión ya no se pierden datos en el cruce tipo `inner join`. 